In [ ]:
# Импорт библиотек
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np
import zipfile

In [ ]:
# Загрузка данных 
with zipfile.ZipFile("titanic.zip") as z:
   # open the csv file in the dataset
    with z.open("train.csv") as f:
        # read the dataset
        train = pd.read_csv(f)
    with z.open("test.csv") as f:
        # read the dataset
        test = pd.read_csv(f)

In [30]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:

test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
# Есть столбцы с пропущенными значениями. Есть текстовые значения.
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Входные данные не нормализованы
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
#Удаляем лишние столбцы. Выделяем предсказываемую переменную. 
X=train.drop(['PassengerId','Survived','Name','Ticket','Cabin'], axis=1)
y=train['Survived']
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [ ]:
#Задаем разбиение множества на обучающую выборку и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Создаем предобработчик 
categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Обработка числовых переменных
numeric_features = ['Age', 'Fare','SibSp','Parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 7)),
    ('scaler', StandardScaler())])

# Итоговый Предобработчик, который пойдет в Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)])

In [ ]:
# Составляем полную цепочку для классификации K-соседей
pipeline_KNN = Pipeline(steps=[('preprocessor', preprocessor),
                      ('knn', KNeighborsClassifier())])

# Задаем таблицу гиперпараметров KNN классификации
params_KNN = {"knn__weights":['uniform', 'distance'],
              "knn__n_neighbors": np.arange(3,20),
              "knn__algorithm": ["auto", "ball_tree", "kd_tree", "brute"]
                 }

# Модель для выбора гиперпараметров и прогноза
tuning_KNN = GridSearchCV(pipeline_KNN, param_grid=params_KNN, cv=5)

# Ищем наилучшее сочетание гиперпараметров полным перебором на обучающей выборке
tuning_KNN.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex',
                                                                          'Pclass']),


In [ ]:
print("Лучшие гиперпараметры Логистической регрессии: {}, Accuracy: {}".\
      format(tuning_KNN.best_params_, tuning_KNN.score(X_test, y_test)))

Лучшие гиперпараметры Логистической регрессии: {'knn__algorithm': 'auto', 'knn__n_neighbors': 16, 'knn__weights': 'uniform'}, Accuracy: 0.7835820895522388


In [ ]:
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
    pred = tuning_KNN.predict(X)
    print(label)
    print(f'Accuracy_{label}={accuracy_score(y, pred):.2f}  \t- доля точных предсказаний классов')
    print(f'Precision_{label}={precision_score(y, pred):.2f} \t- доля правильно предсказаных выживших к предсказанным выжившим')
    print(f'Recall_{label}={recall_score(y, pred):.2f} \t- доля правильно предсказаных выживших к фактически выжившим')
    print(f'F1_{label}={f1_score(y, pred):.2f}      \t- комбинация (среднее гармоническое) Precision и Recall')
    print(f'AUC_ROC_{label}={roc_auc_score(y, pred):.2f} \t- площадь под кривой')
    print()

train
Accuracy_train=0.83  	- доля точных предсказаний классов
Precision_train=0.87 	- доля правильно предсказаных выживших к предсказанным выжившим
Recall_train=0.63 	- доля правильно предсказаных выживших к фактически выжившим
F1_train=0.73      	- комбинация (среднее гармоническое) Precision и Recall
AUC_ROC_train=0.79 	- площадь под кривой

test
Accuracy_test=0.78  	- доля точных предсказаний классов
Precision_test=0.81 	- доля правильно предсказаных выживших к предсказанным выжившим
Recall_test=0.62 	- доля правильно предсказаных выживших к фактически выжившим
F1_test=0.70      	- комбинация (среднее гармоническое) Precision и Recall
AUC_ROC_test=0.76 	- площадь под кривой



In [32]:
pipeline_LogReg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('logreg', LogisticRegression())])

params_LogReg = {"logreg__solver": ["newton-cg", "saga", "lbfgs"],
         "logreg__C": [0.0001, 0.001, 0.01, 0.1, 1.0, 5.0, 10.0] 
         }
tuning_LogReg = GridSearchCV(pipeline_LogReg, param_grid=params_LogReg, cv=5)
tuning_LogReg.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex',
                                                                          'Pclass']),


In [33]:
# Лучшее сочетание параметров обученной модели
print("Лучшие гиперпараметры Логистической регрессии: {}, Accuracy: {}".\
      format(tuning_LogReg.best_params_, tuning_LogReg.score(X_test, y_test)))

Лучшие гиперпараметры Логистической регрессии: {'logreg__C': 1.0, 'logreg__solver': 'newton-cg'}, Accuracy: 0.8059701492537313


In [31]:
# Метрики классификации для обучающего множества, на котором считали кросс-валидацию и тестого, которое мы не трогали
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
    pred = tuning_LogReg.predict(X)
    print(label)
    print(f'Accuracy_{label}={accuracy_score(y, pred):.2f}  \t- доля точных предсказаний классов')
    print(f'Precision_{label}={precision_score(y, pred):.2f} \t- доля правильно предсказаных выживших к предсказанным выжившим')
    print(f'Recall_{label}={recall_score(y, pred):.2f} \t- доля правильно предсказаных выживших к фактически выжившим')
    print(f'F1_{label}={f1_score(y, pred):.2f}      \t- комбинация (среднее гармоническое) Precision и Recall')
    print(f'AUC_ROC_{label}={roc_auc_score(y, pred):.2f} \t- площадь под кривой')
    print()

train
Accuracy_train=0.81  	- доля точных предсказаний классов
Precision_train=0.78 	- доля правильно предсказаных выживших к предсказанным выжившим
Recall_train=0.68 	- доля правильно предсказаных выживших к фактически выжившим
F1_train=0.73      	- комбинация (среднее гармоническое) Precision и Recall
AUC_ROC_train=0.79 	- площадь под кривой

test
Accuracy_test=0.81  	- доля точных предсказаний классов
Precision_test=0.80 	- доля правильно предсказаных выживших к предсказанным выжившим
Recall_test=0.71 	- доля правильно предсказаных выживших к фактически выжившим
F1_test=0.75      	- комбинация (среднее гармоническое) Precision и Recall
AUC_ROC_test=0.79 	- площадь под кривой




Логистическая регрессия лучше из-за то, что точность на обучающей выборке у KNN выше, точность на тестовой лучше у LogReg. Об этом свидетельствует и ROC_AUC - он ведет себя более стабильно у LogReg. 
В целом LogReg имеет меньше изменений точных характеристик на обучающей и тестовой выборках, чем KNN.